In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

# Import pandas and read the charity_data.csv from the provided cloud URL.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'
application_df = application_df.drop(columns=["EIN", "NAME"])

# Display the first few rows to confirm the changes
application_df.head()


,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Determine the number of unique values in each column
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
application_df["APPLICATION_TYPE"].value_counts()

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
application_type_counts = application_df["APPLICATION_TYPE"].value_counts()

# Define the threshold for infrequent categories
threshold = 500  

# Identify application types that occur less than the threshold
application_types_to_replace = application_type_counts[application_type_counts < threshold].index

# Replace in the dataframe
for app in application_types_to_replace:
    application_df["APPLICATION_TYPE"] = application_df["APPLICATION_TYPE"].replace(app, "Other")

# Check to make sure replacement was successful
application_df["APPLICATION_TYPE"].value_counts()


APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts to identify rare categories
classification_counts = application_df["CLASSIFICATION"].value_counts()

# Define a threshold for rare categories
threshold = 1000  

# Identify classifications to replace
classifications_to_replace = classification_counts[classification_counts < threshold].index

# Replace in the dataframe
for cls in classifications_to_replace:
    application_df["CLASSIFICATION"] = application_df["CLASSIFICATION"].replace(cls, "Other")

# Check to make sure replacement was successful
application_df["CLASSIFICATION"].value_counts()


CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [7]:
# Convert categorical data to numeric using one-hot encoding
application_df = pd.get_dummies(application_df)

# Display the first few rows to confirm encoding
application_df.head()


,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
1,1,108590,1,False,False,False,True,False,False,False,...,True,False,False,False,False,False,False,False,True,False
2,1,5000,0,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,True,False
3,1,6692,1,False,False,False,True,False,False,False,...,False,True,False,False,False,False,False,False,True,False
4,1,142590,1,False,False,False,True,False,False,False,...,False,False,True,False,False,False,False,False,True,False


In [8]:
# Split our preprocessed data into our features (X) and target (y) arrays
X = application_df.drop(columns=["IS_SUCCESSFUL"])  # Features
y = application_df["IS_SUCCESSFUL"]  # Target

# Display first few rows of X and y to confirm
X.head(), y.head()


(   STATUS  ASK_AMT  APPLICATION_TYPE_Other  APPLICATION_TYPE_T10  \
 0       1     5000                   False                  True   
 1       1   108590                   False                 False   
 2       1     5000                   False                 False   
 3       1     6692                   False                 False   
 4       1   142590                   False                 False   
 
    APPLICATION_TYPE_T19  APPLICATION_TYPE_T3  APPLICATION_TYPE_T4  \
 0                 False                False                False   
 1                 False                 True                False   
 2                 False                False                False   
 3                 False                 True                False   
 4                 False                 True                False   
 
    APPLICATION_TYPE_T5  APPLICATION_TYPE_T6  APPLICATION_TYPE_T7  ...  \
 0                False                False                False  ...   
 1            

In [9]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

# Display the shapes of the resulting datasets to confirm the split
X_train.shape, X_test.shape, y_train.shape, y_test.shape


((25724, 43), (8575, 43), (25724,), (8575,))

In [10]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler to the training data
X_train_scaled = scaler.fit_transform(X_train)

# Scale the testing data using the same fitted scaler
X_test_scaled = scaler.transform(X_test)

# Display the first few rows of the scaled training data
X_train_scaled[:5]

array([[ 1.39430557e-02, -3.34843874e-02, -8.91865111e-02,
        -1.26474869e-01, -1.80310401e-01, -1.93323172e+00,
        -2.17211760e-01, -1.88868073e-01,  5.27316518e+00,
        -1.44900757e-01, -1.47122472e-01,  1.08678841e+00,
        -4.50061455e-02, -1.07924659e+00, -3.18080475e-02,
        -6.23503924e-03, -1.97203755e-02, -1.00913832e+00,
        -4.05336896e-01,  2.15062580e+00, -2.41986076e-01,
        -2.43601984e-01, -2.64164562e-01, -1.08076349e-01,
        -6.21563724e-02, -8.81784846e-03, -2.13840836e+00,
         2.25801349e+00, -6.52037081e-01, -1.17446358e-01,
        -3.35950033e-02,  6.75069188e-01,  6.37957757e-01,
        -1.48495575e-01, -1.26949951e-01, -3.29185563e-01,
        -8.34734853e-02, -1.69813249e-01, -3.51865775e-01,
        -6.40196894e-02, -6.98784891e-02,  2.78942470e-02,
        -2.78942470e-02],
       [ 1.39430557e-02, -3.34843874e-02, -8.91865111e-02,
        -1.26474869e-01, -1.80310401e-01,  5.17268565e-01,
        -2.17211760e-01, -1.88

# First Trial : This trial uses a neural network with three hidden layers using ReLU activation.


In [11]:
import tensorflow as tf

# Define the optimized model
optimized_nn = tf.keras.Sequential()

# First hidden layer with increased neurons
optimized_nn.add(tf.keras.layers.Dense(units=100, activation="relu", input_dim=X_train_scaled.shape[1]))

# Second hidden layer with more neurons
optimized_nn.add(tf.keras.layers.Dense(units=50, activation="relu"))

# Third hidden layer (newly added)
optimized_nn.add(tf.keras.layers.Dense(units=25, activation="relu"))

# Output layer
optimized_nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Display model summary
optimized_nn.summary()


C:\Users\somay\anaconda3\envs\dev\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 100)                 │           4,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 50)                  │           5,050 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 25)                  │           1,275 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │              26 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 10,751 (42.00 KB)

 Trainable params: 10,751 (42.00 KB)

 Non-trainable params: 0 (0.00 B)

In [15]:
# Compile the optimized model
optimized_nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [16]:
# Train the optimized model
fit_optimized_model = optimized_nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.7019 - loss: 0.5910
Epoch 2/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7313 - loss: 0.5546
Epoch 3/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7303 - loss: 0.5540
Epoch 4/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7289 - loss: 0.5526
Epoch 5/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.7363 - loss: 0.5489
Epoch 6/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.7363 - loss: 0.5431
Epoch 7/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.7335 - loss: 0.5494
Epoch 8/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.7331 - loss: 0.5449
Epoch 9/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.7410 - loss: 0.5388
Epoch 10/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7328 - loss: 0.5474
Epoch 11/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7347 - loss: 0.5457
Epoch 12/100
804/804 ━━━━━━━━━━━━━━━━━━━━

In [17]:
# Evaluate the optimized model
model_loss, model_accuracy = optimized_nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - 5ms/step - accuracy: 0.7263 - loss: 0.5608
Loss: 0.5608101487159729, Accuracy: 0.7262973785400391


In [42]:
# Save the trained model
optimized_nn.save("first_trial_model.h5")

## Second Trial : This trial uses a neural network with different activation functions for hidden layers.


In [18]:
# Define the optimized model (New Trial)
new_optimized_nn = tf.keras.Sequential()

# First hidden layer with different activation function
new_optimized_nn.add(tf.keras.layers.Dense(units=100, activation="tanh", input_dim=X_train_scaled.shape[1]))

# Second hidden layer with more neurons
new_optimized_nn.add(tf.keras.layers.Dense(units=50, activation="selu"))

# Third hidden layer (newly added)
new_optimized_nn.add(tf.keras.layers.Dense(units=25, activation="relu"))

# Output Layer
new_optimized_nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Display model summary
new_optimized_nn.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                      │ (None, 100)                 │           4,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 50)                  │           5,050 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 25)                  │           1,275 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 1)                   │              26 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 10,751 (42.00 KB)

 Trainable params: 10,751 (42.00 KB)

 Non-trainable params: 0 (0.00 B)

In [19]:
# Compile the optimized model
new_optimized_nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model (increase epochs if needed)
fit_model = new_optimized_nn.fit(X_train_scaled, y_train, epochs=150)


Epoch 1/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.7134 - loss: 0.5788
Epoch 2/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.7227 - loss: 0.5619
Epoch 3/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.7292 - loss: 0.5542
Epoch 4/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7297 - loss: 0.5527
Epoch 5/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7343 - loss: 0.5489
Epoch 6/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7366 - loss: 0.5462
Epoch 7/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7340 - loss: 0.5443
Epoch 8/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7304 - loss: 0.5505
Epoch 9/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7317 - loss: 0.5499
Epoch 10/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.7373 - loss: 0.5474
Epoch 11/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.7315 - loss: 0.5485
Epoch 12/150
804/804 ━━━━━━━━━━━━━━━━━━━━

In [20]:
# Evaluate the optimized model on test data
model_loss, model_accuracy = new_optimized_nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - 3ms/step - accuracy: 0.7252 - loss: 0.5678
Loss: 0.5677587389945984, Accuracy: 0.7252478003501892


In [43]:
# Save the trained model
new_optimized_nn.save("second_trial_model.h5")

## Third Trial : This trial uses a neural network with increased neurons, an additional layer, and Leaky ReLU activation.


In [21]:
from tensorflow.keras.layers import LeakyReLU

# Define an improved model with more neurons, an additional layer, and LeakyReLU
new_optimized_nn = tf.keras.Sequential()

# First hidden layer - Increase neurons and use 'tanh'
new_optimized_nn.add(tf.keras.layers.Dense(units=150, activation="tanh", input_dim=X_train_scaled.shape[1]))

# Second hidden layer - More neurons and use LeakyReLU
new_optimized_nn.add(tf.keras.layers.Dense(units=80))
new_optimized_nn.add(LeakyReLU(alpha=0.01))  # Adding LeakyReLU activation

# Third hidden layer - Increase neurons and use 'tanh'
new_optimized_nn.add(tf.keras.layers.Dense(units=50, activation="tanh"))

# Fourth hidden layer (Newly added) - Use LeakyReLU
new_optimized_nn.add(tf.keras.layers.Dense(units=25))
new_optimized_nn.add(LeakyReLU(alpha=0.01))  # Adding LeakyReLU activation

# Output layer
new_optimized_nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Display model summary
new_optimized_nn.summary()


C:\Users\somay\anaconda3\envs\dev\lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_8 (Dense)                      │ (None, 150)                 │           6,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 80)                  │          12,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu (LeakyReLU)              │ (None, 80)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ (None, 50)                  │           4,050 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ (None, 25)                  │           1,275 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_1 (LeakyReLU)            │ (None, 25)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_12 (Dense)                     │ (None, 1)                   │              26 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 24,031 (93.87 KB)

 Trainable params: 24,031 (93.87 KB)

 Non-trainable params: 0 (0.00 B)

In [22]:
# Compile the new optimized model
new_optimized_nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [23]:
# Train the new optimized model
fit_new_model = new_optimized_nn.fit(X_train_scaled, y_train, epochs=150)

Epoch 1/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.7103 - loss: 0.5803
Epoch 2/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.7271 - loss: 0.5593
Epoch 3/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.7316 - loss: 0.5545
Epoch 4/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.7333 - loss: 0.5510
Epoch 5/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.7356 - loss: 0.5487
Epoch 6/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.7344 - loss: 0.5483
Epoch 7/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.7370 - loss: 0.5449
Epoch 8/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.7310 - loss: 0.5496
Epoch 9/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.7364 - loss: 0.5433
Epoch 10/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.7439 - loss: 0.5370
Epoch 11/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.7349 - loss: 0.5463
Epoch 12/150
804/804 ━━━━━━━━━━━━━━━━━━━━

In [24]:
# Evaluate the new optimized model on test data
model_loss, model_accuracy = new_optimized_nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - 3ms/step - accuracy: 0.7263 - loss: 0.5663
Loss: 0.5663431882858276, Accuracy: 0.7262973785400391


In [44]:
# Save the trained model
new_optimized_nn.save("third_trial_model.h5")

## Fourth Trial: This trial focuses on adjusting the input data by removing columns with very low or negative correlation to the target variable.


In [25]:
# Check correlation of features with the target variable
correlations = application_df.corr()
correlations["IS_SUCCESSFUL"].sort_values(ascending=False)

IS_SUCCESSFUL                   1.000000
AFFILIATION_Independent         0.368322
ORGANIZATION_Trust              0.162164
CLASSIFICATION_Other            0.111202
APPLICATION_TYPE_T5             0.088575
APPLICATION_TYPE_T10            0.086805
APPLICATION_TYPE_T6             0.079817
INCOME_AMT_1-9999               0.067058
CLASSIFICATION_C1000            0.062589
ORGANIZATION_Co-operative       0.053520
APPLICATION_TYPE_Other          0.045165
INCOME_AMT_10000-24999          0.036937
INCOME_AMT_100000-499999        0.030542
INCOME_AMT_25000-99999          0.026239
USE_CASE_Preservation           0.024145
ORGANIZATION_Corporation        0.016690
AFFILIATION_Other               0.010121
APPLICATION_TYPE_T7             0.009752
USE_CASE_Other                  0.008765
SPECIAL_CONSIDERATIONS_Y        0.005469
AFFILIATION_National            0.004581
AFFILIATION_Regional            0.003238
AFFILIATION_Family/Parent       0.002608
CLASSIFICATION_C2000           -0.000434
APPLICATION_TYPE

In [27]:
# Display available columns in the DataFrame
print(application_df.columns)

Index(['STATUS', 'ASK_AMT', 'IS_SUCCESSFUL', 'APPLICATION_TYPE_Other',
       'APPLICATION_TYPE_T10', 'APPLICATION_TYPE_T19', 'APPLICATION_TYPE_T3',
       'APPLICATION_TYPE_T4', 'APPLICATION_TYPE_T5', 'APPLICATION_TYPE_T6',
       'APPLICATION_TYPE_T7', 'APPLICATION_TYPE_T8',
       'AFFILIATION_CompanySponsored', 'AFFILIATION_Family/Parent',
       'AFFILIATION_Independent', 'AFFILIATION_National', 'AFFILIATION_Other',
       'AFFILIATION_Regional', 'CLASSIFICATION_C1000', 'CLASSIFICATION_C1200',
       'CLASSIFICATION_C2000', 'CLASSIFICATION_C2100', 'CLASSIFICATION_C3000',
       'CLASSIFICATION_Other', 'USE_CASE_CommunityServ', 'USE_CASE_Heathcare',
       'USE_CASE_Other', 'USE_CASE_Preservation', 'USE_CASE_ProductDev',
       'ORGANIZATION_Association', 'ORGANIZATION_Co-operative',
       'ORGANIZATION_Corporation', 'ORGANIZATION_Trust', 'INCOME_AMT_0',
       'INCOME_AMT_1-9999', 'INCOME_AMT_10000-24999',
       'INCOME_AMT_100000-499999', 'INCOME_AMT_10M-50M', 'INCOME_AMT_1M-5M

In [28]:
# List of columns you want to drop
columns_to_drop = [
    "STATUS", "SPECIAL_CONSIDERATIONS_N", "ASK_AMT", "USE_CASE_ProductDev",
    "USE_CASE_Healthcare", "INCOME_AMT_10M-50M", "INCOME_AMT_5M-10M",
    "AFFILIATION_National", "AFFILIATION_Family/Parent", "AFFILIATION_CompanySponsored"
]

# Keep only the columns that exist in the DataFrame
columns_to_drop = [col for col in columns_to_drop if col in application_df.columns]

# Drop the valid columns
application_df = application_df.drop(columns=columns_to_drop)

# Display first few rows after dropping columns
application_df.head()


,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,...,ORGANIZATION_Corporation,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,SPECIAL_CONSIDERATIONS_Y
0,1,False,True,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
1,1,False,False,False,True,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
2,0,False,False,False,False,False,True,False,False,False,...,False,False,True,False,False,False,False,False,False,False
3,1,False,False,False,True,False,False,False,False,False,...,False,True,False,False,True,False,False,False,False,False
4,1,False,False,False,True,False,False,False,False,False,...,False,True,False,False,False,True,False,False,False,False


In [30]:
# Reapply One-Hot Encoding to convert categorical data into numerical form
application_df = pd.get_dummies(application_df)

# Define the target variable (y) and features (X)
X = application_df.drop(columns=["IS_SUCCESSFUL"])  # Drop target column from features
y = application_df["IS_SUCCESSFUL"]  # Define target variable

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)  # Set random state for reproducibility

# Scale the feature data using StandardScaler
scaler = StandardScaler()  # Initialize the scaler
X_train_scaled = scaler.fit_transform(X_train)  # Fit and transform training data
X_test_scaled = scaler.transform(X_test)  # Transform test data using the same scaler


In [31]:
# Define a new improved model with adjusted layers and activation functions
new_optimized_nn = tf.keras.Sequential()

# First hidden layer - increased neurons and using 'tanh'
new_optimized_nn.add(tf.keras.layers.Dense(units=150, activation="tanh", input_dim=X_train_scaled.shape[1]))

# Second hidden layer - more neurons and using LeakyReLU
new_optimized_nn.add(tf.keras.layers.Dense(units=80))
new_optimized_nn.add(LeakyReLU(alpha=0.01))  # Adding LeakyReLU activation

# Third hidden layer - using 'tanh'
new_optimized_nn.add(tf.keras.layers.Dense(units=50, activation="tanh"))

# Fourth hidden layer - using LeakyReLU
new_optimized_nn.add(tf.keras.layers.Dense(units=25))
new_optimized_nn.add(LeakyReLU(alpha=0.01))  # Adding LeakyReLU activation

# Output layer
new_optimized_nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Display model summary
new_optimized_nn.summary()

C:\Users\somay\anaconda3\envs\dev\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\somay\anaconda3\envs\dev\lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_13 (Dense)                     │ (None, 150)                 │           5,250 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_14 (Dense)                     │ (None, 80)                  │          12,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_2 (LeakyReLU)            │ (None, 80)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_15 (Dense)                     │ (None, 50)                  │           4,050 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_16 (Dense)                     │ (None, 25)                  │           1,275 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_3 (LeakyReLU)            │ (None, 25)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_17 (Dense)                     │ (None, 1)                   │              26 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 22,681 (88.60 KB)

 Trainable params: 22,681 (88.60 KB)

 Non-trainable params: 0 (0.00 B)

In [32]:
# Compile the new optimized model
new_optimized_nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [33]:
# Train the new optimized model
new_optimized_nn.fit(X_train_scaled, y_train, epochs=150, batch_size=32)

Epoch 1/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.7170 - loss: 0.5751
Epoch 2/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.7234 - loss: 0.5640
Epoch 3/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7285 - loss: 0.5569
Epoch 4/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.7318 - loss: 0.5529
Epoch 5/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.7285 - loss: 0.5518
Epoch 6/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.7348 - loss: 0.5475
Epoch 7/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.7336 - loss: 0.5489
Epoch 8/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.7304 - loss: 0.5521
Epoch 9/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.7308 - loss: 0.5490
Epoch 10/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.7331 - loss: 0.5466
Epoch 11/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.7305 - loss: 0.5476
Epoch 12/150
804/804 ━━━━━━━━━━━━━━━━━━━━

In [34]:
# Evaluate the optimized model on test data
model_loss, model_accuracy = new_optimized_nn.evaluate(X_test_scaled, y_test, verbose=2)

# Print the results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - 3ms/step - accuracy: 0.7259 - loss: 0.5607
Loss: 0.5606738328933716, Accuracy: 0.7259474992752075


In [45]:
# Save the trained model
new_optimized_nn.save("fourth_trial_model.h5")

## Fifth Trial: This trial applies PCA to reduce dimensionality, keeping 95% of the variance, followed by a neural network with three hidden layers using ReLU activation.


In [36]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Normalize the dataset
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)  # X should be the feature set without the target variable

# Apply PCA
pca = PCA(n_components=0.95)  # Keeps 95% of the variance
X_pca = pca.fit_transform(X_scaled)

# Split the transformed data
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, random_state=78)

In [37]:
# Define the neural network model
pca_nn = tf.keras.Sequential()

# Input layer
pca_nn.add(tf.keras.layers.Dense(units=100, activation="relu", input_dim=X_train.shape[1]))

# Hidden layers
pca_nn.add(tf.keras.layers.Dense(units=50, activation="relu"))
pca_nn.add(tf.keras.layers.Dense(units=25, activation="relu"))

# Output layer
pca_nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

C:\Users\somay\anaconda3\envs\dev\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [38]:
# Compile the model
pca_nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [39]:
# Train the model
history = pca_nn.fit(X_train, y_train, epochs=150, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.6721 - loss: 0.6235 - val_accuracy: 0.7207 - val_loss: 0.5716
Epoch 2/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7247 - loss: 0.5627 - val_accuracy: 0.7210 - val_loss: 0.5620
Epoch 3/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7242 - loss: 0.5625 - val_accuracy: 0.7219 - val_loss: 0.5609
Epoch 4/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7286 - loss: 0.5554 - val_accuracy: 0.7236 - val_loss: 0.5595
Epoch 5/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.7311 - loss: 0.5545 - val_accuracy: 0.7245 - val_loss: 0.5563
Epoch 6/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.7328 - loss: 0.5511 - val_accuracy: 0.7242 - val_loss: 0.5557
Epoch 7/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.7381 - loss: 0.5443 - val_accuracy: 0.7254 - val_loss: 0.5566
Epoch 8/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.7334 - loss: 0.5528 - val_accu

In [40]:
# Evaluate the model
model_loss, model_accuracy = pca_nn.evaluate(X_test, y_test, verbose=2)

# Print results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - 3ms/step - accuracy: 0.7259 - loss: 0.5610
Loss: 0.5610270500183105, Accuracy: 0.7259474992752075


In [46]:
# Save the trained model
new_optimized_nn.save("fifth_trial_model.h5")